# 1.1: Framework Selection

The source and target frameworks for `ivy.unify`, `ivy.compile` and `ivy.transpile` can be: (a) inferred from the arguments and/or inspection of the function, (b) specified globally or (c) specified locally. All examples in the [Building Blocks]() either infer the source and target frameworks or specify them globally (via `ivy.set_backend`). We'll explore these various options, and also explore which modes take priority. For these examples, all functions are called *eagerly*. Please go through the [Lazy vs Eager]() notebook if you haven't already.

## Unify

Consider again this simple `torch` function:

In [ ]:
import ivy
import torch

def normalize(x, mean, std):
    return torch.div(torch.sub(x, mean), std)

Let's also create the dummy data as before:

In [ ]:
# import NumPy
import numpy as np

# create random NumPy arrays for testing
x = np.randon.uniform(size=10)
mean = np.mean(x)
std = np.std(x)

This time, let's assume that our target framework is JAX:

In [ ]:
import jax.numpy as jnp

x = jnp.array(x)
mean = jnp.array(mean)
std = jnp.array(std)

In the example below, the *source* framework of `torch` is *inferred* from the *function* `normalize`.

In [ ]:
norm = ivy.unify(normalize, args=(x, mean, std))

As mentioned in the [Unify]() notebook, `ivy.unify` is able to detect that the original `normalize` function is implemented in PyTorch by using the `inspection` module. `ivy.unify` then converts the framework-specific PyTorch implementation into a framework-agnostic Ivy implementation, which is compatible with all frameworks.

For some functions, this would not be possible. Consider the example below:

In [ ]:
def normalize_via_operators(x, mean, std):
    return (x - mean) / std

There is no way to determine the source framework from this function via the `inspection` module. This code uses built-in operators only, which are compatible with all ML frameworks. You might therefore think "this is already unified", but that's not true. Every ML framework has its own unique rules for broadcasting shapes and data types for elementwise functions, which must all be taken into account when converting code to Ivy.

Rather than inferring the framework, the framework can be specified *locally* as follows:

In [ ]:
norm = ivy.unify(normalize_via_operators, args=(x, mean, std), from="torch")

Note that in all of the examples above, the arguments are in fact `jax` arrays. During function tracing, the `jax` arrays are converted to `torch` tensors automatically.

## Compile

In the example below, the *target* framework of `jax` is *inferred* from the *arguments*.

In [ ]:
norm_comp = ivy.compile(norm, args=(x, mean, std))

However, if the Ivy function `norm` was purely generative (not consuming any arrays in the input), then this would not be possible. In such cases, we could set the target framework globally like so. If the type of the arguments conflicts with the globally set backend, then an error will be thrown.

In [ ]:
ivy.set_backend("jax")
norm_comp = ivy.compile(norm, args=(x, mean, std))

Finally, the target framework can be provided locally. This will override any globally set backend, but again the arguments must be of the correct type in order to avoid errors.

In [ ]:
ivy.set_backend("tensorflow") # a different global backend
norm_comp = ivy.compile(norm, args=(x, mean, std), to="jax") # doesn't matter, jax specified locally

## Transpile

All consideration for both `ivy.unify` and `ivy.compile` are combined for `ivy.transpile`, which is effectively shorthand for the combination of these two functions (as explained in the [Transpile]() section).

In the example below, the *source* framework of `torch` is *inferred* from the *function* `normalize`, **and** the *target* framework of `jax` is *inferred* from the *arguments*.

In [ ]:
norm = ivy.transpile(normalize, args=(x, mean, std))

In the example below, the *source* framework is specified *locally* (would be necessary if transpiling `normalize_via_operators` for example) **and** the *target* framework of `jax` is *inferred* from the *arguments*.

In [ ]:
norm = ivy.transpile(normalize, args=(x, mean, std), from="torch")

In the example below, the *source* framework is specified *locally* **and** the *target* framework of `jax` is specified *globally*. This might be necessary if there are no array arguments for the function.

In [ ]:
ivy.set_backend("jax")
norm = ivy.transpile(normalize, args=(x, mean, std), from="torch")

As with `ivy.compile`, the target framework can be provided locally. This will override any globally set backend, but again the arguments must be of the correct type in order to avoid errors.

In the example below, the *source* framework is specified *locally* **and** the *target* framework of `jax` is also specified *locally*. Again, this might be necessary if there are no array arguments for the function.

In [ ]:
ivy.set_backend("tensorflow") # a different global backend
norm = ivy.transpile(normalize, args=(x, mean, std), from="torch", to="jax") # doesn't matter, jax specified locally

## Round Up

That's it, you now know the difference between inferring, locally specifying, and globally specifying source and target frameworks for `ivy.unify`, `ivy.compile` and `ivy.transpile`! However, there are several other important topics to master before you're ready to unify ML code like a pro 🥷. Next, we'll be exploring how these three functions can all be called as function decorators!